In [7]:
# necessary libs to run deepod and pyod
!pip install pyod
!pip install deepod
!pip install numpy==1.24.4 --force-reinstall

  Using cached numpy-1.24.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.6 kB)
Using cached numpy-1.24.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.4
    Uninstalling numpy-1.24.4:
      Successfully uninstalled numpy-1.24.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
torchvision 0.21.0+cu124 requires torch==2.6.0, but you have torch 1.13.0 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.24.4 which is incompatible.
treescope 0.1.9 requires numpy>=1.25.2, but you have numpy 1.24.4 which is incompatible.
pymc 5.22.0 requires numpy>=1.25.0, but you have numpy 1.24.4 which is incompatible.
accelerate 1.6.0 requires torch

  Using cached pyod-2.0.5-py3-none-any.whl.metadata (46 kB)
Using cached pyod-2.0.5-py3-none-any.whl (200 kB)


In [2]:
running='colab'

In [3]:
import os

In [6]:
if running=='local':
    os.chdir(os.path.dirname(os.getcwd()))
    project_path = os.getcwd()
elif running=='colab':
    from google.colab import drive
    drive.mount('/content/drive')
    project_path_colab = "/content/drive/MyDrive/Projeto ML/2025/AD/second_setup"
    repo_name = "adaptative-text-anomaly-detection"
    project_path = os.path.join(project_path_colab, repo_name)
    os.chdir(project_path)
else:
    raise ValueError("Invalid running environment. Choose 'local' or 'colab'.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import os
import sys
import pandas as pd
sys.path.append(r'../src')
sys.path.append(r'./src')

import time
import os
from tqdm import tqdm
import os

import pandas as pd

In [8]:
from pipeline.anomaly_detection import label_normal_vs_anomaly,adjust_contamination, pretty_print_data_info, split_data
from pipeline.benchmark_runner import benchmark_unsupervised_models, benchmark_semisupervised_models
from models.MLP import MLP

In [9]:
from deepod.models import DeepSAD, DeepSVDD, DevNet
from pyod.models.xgbod import XGBOD
from pyod.models.lof import LOF
from pyod.models.iforest import IForest
from pyod.models.xgbod import XGBOD
from pyod.models.ocsvm import OCSVM
from pyod.models.auto_encoder import AutoEncoder
from pyod.models.vae import VAE
from pyod.models.pca import PCA
from pyod.models.kde import KDE
from pyod.models.hbos import HBOS
from sklearn.svm import OneClassSVM
from sklearn.neural_network import MLPClassifier

In [ ]:
config = {
    "en": {
        "datasets": [
            "tweets-hate-speech-detection/tweets_hate_speech_detection",
            "SetFit/20_newsgroups",
            "cardiffnlp/tweet_eval"
        ],
        "encoders": [
            "sentence-transformers/distiluse-base-multilingual-cased-v1",
            "sentence-transformers/distiluse-base-multilingual-cased-v2",
            "FacebookAI/xlm-roberta-large"
        ]
    },
    "pt": {
        "datasets": [
            "JAugusto97/told-br",
            "wikinews",
            "augustop/portuguese-tweets-for-sentiment-analysis"
        ],
        "encoders": [
            "sentence-transformers/distiluse-base-multilingual-cased-v1",
            "sentence-transformers/distiluse-base-multilingual-cased-v2",
            "FacebookAI/xlm-roberta-large",
            "neuralmind/bert-base-portuguese-cased",
            "neuralmind/bert-large-portuguese-cased",
            "PORTULAN/serafim-100m-portuguese-pt-sentence-encoder-ir"
        ]
    }
}

model_groups = {
    "semi": {
        "models": {
            "DevNet": lambda: DevNet(),
            "DeepSAD": lambda: DeepSAD(epochs=100, rep_dim=128, device='cuda'),
            "XGBOD": lambda: XGBOD(estimator_list=[LOF(), IForest()]),
            "MLP": lambda: MLP()
        },
        "benchmark_fn": benchmark_semisupervised_models,
        "extra_args": {
            "contamination_levels": [0.05],
            "n_rounds": 1
        },
        "wrap_model": lambda name, fn: {name: (fn, False)}  # unpack into (model_fn, uncertainty)
    },
    "unsupervised": {
        "models": {
            "IForest": lambda: IForest(),
            "LOF": lambda: LOF(),
            "DeepSVDD": lambda: DeepSVDD(epochs=100, rep_dim=128),
            "OCSVM": lambda: OCSVM(kernel='rbf', nu=0.05, gamma='scale'),
            "OneClassSVM": lambda: OneClassSVM(kernel='rbf', nu=0.05, gamma='scale'),
            "AutoEncoder": lambda: AutoEncoder(),
            "VAE": lambda: VAE(),
            "HBOS": lambda: HBOS()
        },
        "benchmark_fn": benchmark_unsupervised_models,
        "extra_args": {},
        "wrap_model": lambda name, fn: {name: fn}  # no need to wrap with (fn, uncertainty)
    }
}

In [19]:
total_combinations = sum(
    len(cfg["datasets"]) * len(cfg["encoders"])
    for cfg in config.values()
)

with tqdm(total=total_combinations, desc="Benchmarking dataset/encoder pairs") as pbar:
    for lang, lang_config in config.items():
        for dataset_name in lang_config["datasets"]:
            dataset_short = dataset_name.split("/")[-1]

            for encoder_name in lang_config["encoders"]:
                encoder_short = encoder_name.split("/")[-1]
                combo_name = f"{dataset_short}_{encoder_short}"

                try:
                    print(f"\nProcessing: {combo_name}")
                    texts_df = pd.read_parquet(os.path.join(project_path, f"data/texts_{dataset_short}.parquet"))
                    labels_df = pd.read_parquet(os.path.join(project_path, f"data/labels_{dataset_short}.parquet"))
                    embeddings_df = pd.read_parquet(os.path.join(project_path, f"data/embeddings_{dataset_short}_{encoder_short}.parquet"))

                    labeled_anomalies_df = label_normal_vs_anomaly(labels_df, as_df=True)

                    texts, labels, embeddings = adjust_contamination(
                        texts=texts_df.squeeze().tolist(),
                        labels=labeled_anomalies_df.squeeze().values,
                        embeddings=embeddings_df.values,
                        perc_anomalous=0.05
                    )

                    pretty_print_data_info(texts, labels, embeddings)
                    x_train, x_test, y_train, y_test = split_data(embeddings, labels, random_state=42)

                    # Loop through model groups (semi + unsupervised)
                    for group_name, group_info in model_groups.items():
                        models = group_info["models"]
                        benchmark_fn = group_info["benchmark_fn"]
                        extra_args = group_info["extra_args"]
                        wrap_model = group_info["wrap_model"]

                        for model_name, model_fn in models.items():
                            print(f"Running {group_name} model: {model_name} on {combo_name}")
                            start_time = time.time()

                            benchmark_fn(
                                x_train, y_train, x_test, y_test,
                                model_constructor=wrap_model(model_name, model_fn),
                                dataset_name=combo_name,
                                **extra_args
                            )

                            print(f"✅ Finished {model_name} in {time.time() - start_time:.2f} seconds")

                except Exception as e:
                    print(f"[❌ ERROR] Failed processing {combo_name}: {e}")

                pbar.update(1)


Benchmarking dataset/encoder pairs:   0%|          | 0/3 [00:00<?, ?it/s]


Processing: tweets_hate_speech_detection_distiluse-base-multilingual-cased-v1
✅ Selected normal label (inlier → 0): 0
⚠️  Anomaly labels (outlier → 1): [-1, 1]
Shape of embeddings: (31206, 512)
Shape of labels: (31206,)
Number of anomalies: 1486
Number of normal: 29720
Percentage of anomalies: 0.047619047619047616
Shape of texts: 31206
Train set: (24964, 512) (with 1189 outliers and 23775 inliers)
Test  set: (6242, 512) (with 297 outliers and 5945 inliers)
Running semi model: DevNet on tweets_hate_speech_detection_distiluse-base-multilingual-cased-v1
→ Running DevNet (uncertainty_model=False)
Start Training...
ensemble size: 1
MLPnet(
  (network): Sequential(
    (0): LinearBlock(
      (linear): Linear(in_features=512, out_features=100, bias=False)
      (act_layer): ReLU()
    )
    (1): LinearBlock(
      (linear): Linear(in_features=100, out_features=50, bias=False)
      (act_layer): ReLU()
    )
    (2): LinearBlock(
      (linear): Linear(in_features=50, out_features=1, bias=Fa


testing: 100%|██████████| 391/391 [00:00<00:00, 1500.57it/s]

testing: 100%|██████████| 391/391 [00:00<00:00, 1335.84it/s]

testing: 100%|██████████| 98/98 [00:00<00:00, 1528.56it/s]


✅ Finished DevNet in 104.47 seconds
Running semi model: DeepSAD on tweets_hate_speech_detection_distiluse-base-multilingual-cased-v1
→ Running DeepSAD (uncertainty_model=False)
Start Training...
ensemble size: 1
training data counter: Counter({0: 23775, -1: 1189})
MLPnet(
  (network): Sequential(
    (0): LinearBlock(
      (linear): Linear(in_features=512, out_features=100, bias=False)
      (act_layer): ReLU()
    )
    (1): LinearBlock(
      (linear): Linear(in_features=100, out_features=50, bias=False)
      (act_layer): ReLU()
    )
    (2): LinearBlock(
      (linear): Linear(in_features=50, out_features=128, bias=False)
      (act_layer): Identity()
    )
  )
)
epoch  1, training loss: 0.854045, time: 0.9s
epoch 10, training loss: 0.307587, time: 0.9s
epoch 20, training loss: 0.184367, time: 0.9s
epoch 30, training loss: 0.265096, time: 0.9s
epoch 40, training loss: 0.160461, time: 0.9s
epoch 50, training loss: 0.103714, time: 0.9s
epoch 60, training loss: 0.204419, time: 1.1s



testing: 100%|██████████| 391/391 [00:00<00:00, 1460.64it/s]

testing: 100%|██████████| 391/391 [00:00<00:00, 1369.03it/s]

testing: 100%|██████████| 98/98 [00:00<00:00, 1254.16it/s]
/usr/local/lib/python3.11/dist-packages/pyod/models/base.py:554: UserWarning: y should not be presented in unsupervised learning.
  warnings.warn(


...

✅ Finished AutoEncoder in 40.00 seconds
Running unsupervised model: VAE on tweets_hate_speech_detection_xlm-roberta-large
→ Running VAE [unsupervised]



Training: 100%|██████████| 30/30 [02:29<00:00,  4.99s/it]
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(


✅ Finished VAE in 152.82 seconds
Running unsupervised model: HBOS on tweets_hate_speech_detection_xlm-roberta-large
→ Running HBOS [unsupervised]


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
Benchmarking dataset/encoder pairs: 100%|██████████| 3/3 [44:52<00:00, 897.35s/it]

✅ Finished HBOS in 6.36 seconds
